# PROJECT 3

**GeoSpatial Data:**
- To show the optimal location for a new business to grow based on parameters:
    - Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design.
    - 30% of the company staff have at least 1 child.
    - Developers like to be near successful tech startups that have raised at least 1 Million dollars.
    - Executives like Starbucks A LOT. Ensure there's a starbucks not too far.
    - Account managers need to travel a lot.
    - Everyone in the company is between 25 and 40, give them some place to go party.
    - The CEO is vegan.
    - If you want to make the maintenance guy happy, a basketball stadium must be around 10 Km.
    - The office dog—"Dobby" needs a hairdresser every month. Ensure there's one not too far away.


In [1]:
#Data:
import json
import requests
import pandas as pd
import numpy as np
from pymongo import MongoClient
import pandas as pd

#Map:
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd

#visualisation:
#import seaborn as sns
#import matplotlib.pyplot as plt
#import matplotlib as mpl
#from cycler import cycler
#import hvplot.pandas
#import lxml.html


In [2]:
client = MongoClient("localhost:27017")
db = client["ironhack"]
c = db.get_collection("companies")
print(c)
db.list_collection_names()
companies = db["companies"]

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ironhack'), 'companies')


In [3]:
pipeline = [
    {
        "$match": {
            "category_code": {
                "$in": [
                    "analytics",
                    "biotech",
                    "cleantech",
                    "ecommerce",
                    "messaging",
                    "mobile",
                    "nanotech",
                    "social",
                    "software",
                    "web",
                    "games_video"
                ]
            },
            "founded_year": {"$gte": 2010}
        }
    },
    {
        "$unwind": "$funding_rounds"
    },
    {
        "$match": {
            "funding_rounds.raised_amount": { "$gte": 1000000 }
        }
    },
    {
        "$group": {
            "_id": "$_id",
            "name": { "$first": "$name" },
            "offices": { "$first": "$offices" },
            "category_code": { "$first": "$category_code" },
            "founded_year": { "$first": "$founded_year" },
            "raised_amount": { "$sum": "$funding_rounds.raised_amount" }
        }
    },
    {
        "$match": {
            "raised_amount": { "$gte": 1000000 }
        }
    }
]

In [4]:
result = list(companies.aggregate(pipeline))
print(len(result))

26


In [5]:
data_tech = pd.DataFrame(result)
data_tech.sample()

,_id,name,offices,category_code,founded_year,raised_amount
13,52cdef7d4bab8bd675299583,Integrate,"[{'description': 'Scottsdale HQ', 'address1': ...",software,2010,34750000


In [21]:
# Create a list to store the data for each office
data = []
for company in result:
    for office in company['offices']:
        data.append({
            'name': company['name'],
            'office description': office['description'],
            'office latitude': office['latitude'],
            'office longitude': office['longitude']
        })

# Convert the list into a Pandas DataFrame
df = pd.DataFrame(data)

# Rearrange the columns in the desired order
df = df[['name', 'office description', 'office latitude', 'office longitude']]

df2 = df.drop(columns=['office description'])
df3 = df2.dropna(subset=['office latitude','office longitude'])
df4 = df3.dropna().reset_index(drop=True)
df4.head()

,name,office latitude,office longitude
0,GENWI,33.817100,-111.903500
1,Pixsta,51.518190,-0.208979
2,AppNeta,42.318138,-71.247560
3,Social Gaming Network,37.446823,-122.161523
4,Social Gaming Network,34.069849,-118.405418


In [22]:
df.to_csv('data/tech_companies.csv', index = False, encoding='utf-8')

In [7]:
tech_heat_map = folium.Map(location=[-3.702818, 40.443921], zoom_start=2)

heat_data = [[row['office latitude'], row['office longitude']] for index, row in df4.iterrows()]

folium.plugins.HeatMap(heat_data, radius=10).add_to(tech_heat_map)

tech_heat_map

In [8]:
pipeline2 = [
    {
        "$match": {
            "category_code": {
                "$in": [
                    "design",
                    "fashion",
                    "advertising"]}}}, 
        {"$group": {
            "_id": "$_id",
            "name": { "$first": "$name" },
            "offices": { "$first": "$offices" },
            "category_code": { "$first": "$category_code" },
        }
    }]

In [9]:
result2 = list(companies.aggregate(pipeline2))
print(len(result2))

942


In [13]:
data_design = pd.DataFrame(result2)
data_design.head()

,_id,name,offices,category_code
0,52cdef7d4bab8bd67529a17d,Neoganda,"[{'description': 'Los Angeles Office', 'addres...",advertising
1,52cdef7d4bab8bd675299d47,hornet,"[{'description': '', 'address1': 'Unit 48/22 B...",advertising
2,52cdef7e4bab8bd67529b8e3,Beaudry Studio,[],advertising
3,52cdef7d4bab8bd67529a23c,Gourmet Ads,"[{'description': '', 'address1': '', 'address2...",advertising
4,52cdef7e4bab8bd67529b5a6,Arisey,"[{'description': '', 'address1': '', 'address2...",advertising


In [17]:
data = []
for company in result2:
    for office in company['offices']:
        data.append({
            'name': company['name'],
            'office description': office['description'],
            'office latitude': office['latitude'],
            'office longitude': office['longitude']
        })

# Convert the list into a Pandas DataFrame
df_design = pd.DataFrame(data)

# Rearrange the columns in the desired order
df_design = df_design[['name', 'office description', 'office latitude', 'office longitude']]

df_design2 = df_design.drop(columns=['office description'])
df_design3 = df_design2.dropna(subset=['office latitude','office longitude'])
df_design4 = df_design3.dropna().reset_index(drop=True)
df_design4.head()

,name,office latitude,office longitude
0,Neoganda,34.076404,-118.375053
1,hornet,-33.863220,151.179693
2,Arisey,40.756054,-73.986951
3,dimension5 labs,35.674675,-105.948434
4,SpotXchange,39.896740,-105.073590


In [23]:
df_design4.to_csv('data/design_companies.csv', index = False, encoding='utf-8')

In [15]:
design_heat_map = folium.Map(location=[-3.702818, 40.443921], zoom_start=2)

heat_data2 = [[row['office latitude'], row['office longitude']] for index, row in df_design4.iterrows()]

folium.plugins.HeatMap(heat_data2, radius=10).add_to(design_heat_map)

design_heat_map